In [23]:
import numpy as np
import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import heapq
import math

In [24]:
devD_df = pd.read_csv('dev/devD.tsv', delimiter='\t', header=None)
trainD_df = pd.read_csv('train/trainD.tsv', delimiter='\t', header=None)

# applying groupby() function to 
# group the data on team value. 
tD_gk = trainD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

trainD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()

trainD_enc = pd.DataFrame(mlb.fit_transform(trainD_df[1]),
                   columns= mlb.classes_, 
                   index= trainD_df.index).max(level=0)

trainD_enc['file'] = ""
for i in range(trainD_enc.shape[0]):
    f = open("train/text_files_en/"+ trainD_enc.index[i] + ".txt", "r")
    trainD_enc['file'][i] = f.read()
    
# applying groupby() function to 
# group the data on team value. 
tD_gk = devD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

devD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()
devD_enc = pd.DataFrame(mlb.fit_transform(devD_df[1]),
                   columns= mlb.classes_, 
                   index= devD_df.index).max(level=0)

devD_enc['file'] = ""
for i in range(devD_enc.shape[0]):
    f = open("dev/text_files_en/"+ devD_enc.index[i] + ".txt", "r")
    devD_enc['file'][i] = f.read()
    
columns_train = trainD_enc.columns
columns_dev = devD_enc.columns

count = 0
for x in columns_dev:
    if x not in columns_train:
        trainD_enc[x] = 0
        
count = 0
for x in columns_train:
    if x not in columns_dev:
        devD_enc[x] = 0 
        
train = trainD_enc['file']
train = pd.DataFrame(train)

del trainD_enc['file']

train['tar'] = trainD_enc.values.tolist()

dev = devD_enc['file']
dev = pd.DataFrame(dev)

del devD_enc['file']

dev['tar'] = devD_enc.values.tolist()

train = pd.concat([train, dev.iloc[:250]])

#dev = dev.iloc[200:]

train = train.reset_index()
#dev = dev.reset_index()

del train[0]
#del dev[0]

train = train.rename(columns={"file": "text", "tar": "labels"})
dev = dev.rename(columns={"file": "text", "tar": "labels", 0:'index'})

/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
with open('labels.txt', 'w') as f:
    for i in trainD_enc.columns:
        f.write("%s\n" % i)

In [27]:
files = os.listdir('test/text_files_en/')
test['files'] = files
test['text'] = ""
for i in range(len(files)):
    f = open('test/text_files_en/'+ files[i], "r")
    test['text'][i] = f.read()    
test.index = test['files']
del test['files']
test

,text
files,
S1139-76322010000100010-1.txt,A 12-year-old boy attended the Primary Care Pe...
S1130-01082008001100010-1.txt,A 57-year-old woman diagnosed at 44 years of a...
S0034-98872014000100018-1.txt,A 53-year-old woman with a history of type 2 d...
S0211-57352014000400010-1.txt,A 60-year-old patient with no relevant psychia...
S0210-48062009001000016-2.txt,A 75-year-old male presented with a history of...
...,...
S1137-66272011000300021-1.txt,A 77-year-old male patient was admitted to the...
S0210-48062005000300008-1.txt,"A 45-year-old patient, with no personal histor..."
S1130-01082008000900008-1.txt,A 28-year-old Ecuadorian woman living in Spain...


In [28]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [29]:
train['text'] = train['text'].apply(lambda x: remove_punctuation(x), 1)
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
train['text'] = train['text'].apply(lambda x: remove_stopwords(x))
train['text'] = train['text'].apply(lambda x: word_lemmatizer(x))
#train['text'] = train['text'].apply(lambda x: word_stemmer(x))

In [30]:
def split_sentences(df, leng, overlap):               # leng is excluding the overlap
    new_df = pd.DataFrame(columns=df.columns)         # new df with same coulmns
    row = 0
    for i in range(df.shape[0]):                      # loop through df rows
        words = df['text'].loc[i].split()
        count = 0
        for j in range(math.ceil(len(words)/leng)):   #loop through sentence
            new_df.loc[row] = df.loc[i]
            new_df['text'].loc[row] = ""
            for k in range (leng + overlap):
                if(j*leng+k < len(words)):
                    new_df['text'].loc[row] += words[j*leng + k] + " "
            row += 1
    return new_df

In [31]:
temp = split_sentences(train, 100, 30)
temp

,text,labels
0,describe case 37yearold man previous active li...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,neurological examination revealed meningeal si...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,transthoracic echocardiography show significan...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,report case 29yearold woman underwent pelvic u...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,cytoplasm well nucleus slightly eosinophilic m...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1784,treated rehabilitation discharge progressive i...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1785,report case 64yearold male hiv infection known...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1786,945 mgdl additionally amino aciduria moderate ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1787,73yearold woman history hypertensive heart dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
models = [['bert', 'bert-base-uncased'], ['roberta', 'roberta-base-uncased'], ['bert', 'emilyalsentzer/Bio_ClinicalBERT']]

In [35]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel(
    "bert", # model name (for bert change to "bert")
    "emilyalsentzer/Bio_ClinicalBERT", # pretrained model (for bert change to "bert-base-uncased")
    num_labels= 2194,
    args={"num_train_epochs": 40, "train_batch_size": 5, "fp16_opt_level" : 'O0',
    "gradient_accumulation_steps": 3, "overwrite_output_dir": True,  "learning_rate": 4e-5,
          "max_seq_length": 150, "save_eval_checkpoints": False, "save_model_every_epoch": False,
         "output_dir": "outputs/",
    "cache_dir": "cache/",
    "best_model_dir": "outputs/best_model/"},
)

In [36]:
%load_ext tensorboard
%tensorboard --logdir {'runs/'}

In [37]:
model.train_model(temp)

Converting to features started. Cache is not used.



Selected optimization level O0:  Pure FP32 training.

Defaults for this optimization level are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0


Running loss: 0.408022


Running loss: 0.104747


Running loss: 0.049130


...

Running loss: 0.040360


Running loss: 0.030488


Running loss: 0.033780


Running loss: 0.028949


Running loss: 0.040485

Training of bert model complete. Saved to outputs/.


In [47]:
predictions, raw_outputs = model.predict(test['text'])

Converting to features started. Cache is not used.


In [62]:
test['labels'] = predictions
test['probabilities'] = raw_outputs.tolist()

In [63]:
test

,text,labels,probabilities
files,,,
S1139-76322010000100010-1.txt,A 12-year-old boy attended the Primary Care Pe...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007645175792276859, 0.004389728885143995, 0..."
S1130-01082008001100010-1.txt,A 57-year-old woman diagnosed at 44 years of a...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007642915938049555, 0.004386870190501213, 0..."
S0034-98872014000100018-1.txt,A 53-year-old woman with a history of type 2 d...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.00764445262029767, 0.004388163797557354, 0...."
S0211-57352014000400010-1.txt,A 60-year-old patient with no relevant psychia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007644980680197477, 0.004388651344925165, 0..."
S0210-48062009001000016-2.txt,A 75-year-old male presented with a history of...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007647050078958273, 0.004390604328364134, 0..."
...,...,...,...
S1137-66272011000300021-1.txt,A 77-year-old male patient was admitted to the...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007645049132406712, 0.004389468114823103, 0..."
S0210-48062005000300008-1.txt,"A 45-year-old patient, with no personal histor...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007650071755051613, 0.004393056035041809, 0..."
S1130-01082008000900008-1.txt,A 28-year-old Ecuadorian woman living in Spain...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007646652404218912, 0.004389616195112467, 0..."


In [64]:
test.to_csv('results.csv', header = True, index = True)